### Init

In [1]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

In [2]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

In [3]:
image_ids = list ( map( int , db.get_image_ids()) )
sorted_image_ids = sorted(image_ids)
print( sorted_image_ids )

[]


In [4]:
from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

In [5]:
from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

In [6]:
from src.lcp.lcp import LCP

In [7]:
lcp = LCP(net, aux_net, dataloader_train)
lcp.init_for_indices()

[LCP] 初始化完成，共 43 層的 channel 索引


In [ ]:
from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information.csv' )
prune_db.initial()

檔案 ./src/db/prune_channel_information.csv 已存在，將刪除並重新創建。
創建新檔案: ./src/db/prune_channel_information.csv
檔案 ./src/db/prune_channel_information.csv 初始化完成。


In [9]:

layers = lcp.get_layers_name()
for name, ch in layers:
    if name == 'layer2.0.conv2' or name == 'layer3.0.conv2':
        pass
    else:
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")
    prune_db.write_data(
        layer = f"net_feature_maps.{name}",
        original_channel_num= len(keep) + len(discard),
        num_of_keep_channel = len(keep),
        keep_index  = keep
    )

layer2.0.conv2: 128 channels
[LCP] 開始基於數學推導的無梯度通道重要性計算 - net_feature_maps.layer2.0.conv2


ValueError: 無有效的特徵統計資料

### Test for Pruner

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )

In [ ]:
print( layers )
layer_names = []
for name, ch in layers:
    if name.endswith('.conv1') or name.endswith('.conv2'):
        layer_names.append( name )

In [ ]:
for idx, layer_name in enumerate(layer_names):
    if idx != 15:
        continue
    lcp.prune_layer(
        layer_name   = layer_name,
        discard_rate = 0.8,
    )
    break

In [ ]:
print( lcp._prune_net )

In [ ]:
from src.lcp.lcpfinetune import LCPFineTune
lcp_finetune = LCPFineTune(
    prune_net = lcp._prune_net,
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg       = cfg,
    optimizer=optimizer,
    parameters=parameters
)

In [ ]:
cfg.defrost()

cfg.train.optim.max_iter = 10
cfg.train.do_training = True
cfg.output.print_iter = 2
cfg.eval.iter = 5
cfg.train.batch_size = 1
cfg.train.batch_size = 1  # 最小批次
cfg.train.accumulate_grad_batches = 1

# 關閉不必要的功能
if hasattr(cfg.train, 'mining'):
    cfg.train.mining.do_mining = False
if hasattr(cfg.output, 'best_model'):
    cfg.output.best_model.do_get_best_model = False

cfg.freeze()
        

In [ ]:
from src.util.loss import LCPFinetuneCriterion
lcp_criterion = LCPFinetuneCriterion(
    original_criterion=criterion,  # 原始損失函數實例
    aux_net=lcp._aux_net,  # 包含 aux_loss 方法的實例
    auxiliary_weight=1.0  # 可以調整輔助損失權重
)

In [ ]:
lcp_finetune.start_finetune(
    criterion= lcp_criterion,  # 使用自定義損失函數
)

In [ ]:
# lcp_finetune.finetune_one_batch(
#     batch_data = data,
#     lcp_instance = lcp,
#     criterion = criterion
# )

In [ ]:
# lcp_finetune.finetune_loop(
#     lcp_instance = lcp,
#     criterion = criterion,
#     dataloaders_eval = dataloaders_eval,
#     max_iters = 5,
#     print_interval=2
# )